In [20]:
import pandas as pd

In [21]:
def to_scientific_notation(value, decimal_places=4):
    """
    숫자를 지정된 소수점 자릿수의 지수 표기법으로 변환합니다.
    """
    format_string = f'{{:.{decimal_places}e}}'
    return format_string.format(value)

def convert_column_to_scientific(df, column_name, decimal_places=4):
    """
    DataFrame의 특정 열을 지수 표기법으로 변환합니다.
    """
    df[column_name] = df[column_name].apply(lambda x: to_scientific_notation(x, decimal_places))
    return df

In [22]:
# general information
file_path = './kinetic_DB/'
i_list = [1, 22]

In [23]:
# bolsigdb.dat 생성
with open('bolsigdb.dat','w') as file:
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        f = convert_column_to_scientific(f, 'Energy(eV)')
        f = convert_column_to_scientific(f, 'Cross_section(m2)')

        type = f['Type'].loc[0]
        reaction = f['Reaction'].loc[0]

        if type == 'ELASTIC' or 'EFFECTIVE':
            eloss = str(f['ELoss'].loc[0])+' / mass ratio'
        else:
            eloss = str(f['ELoss'].loc[0])+' / threshold energy'
        line = '-----------------------------'

        file.write(type+'\n')
        file.write(reaction+'\n')
        file.write(eloss+'\n')
        file.write(line+'\n')
        for j in range(len(f)):
            file.write('  '+str(f['Energy(eV)'].loc[j])+'  '+str(f['Cross_section(m2)'].loc[j])+'\n')
        file.write(line+'\n')
        file.write('\n')
    
    file.close()

In [30]:
# kinep.inp 생성
with open('kinep.inp','w') as file:
    species = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        species += f['Formula'].loc[0].split(' ')
    species = list(set(species))
    species.remove('+')
    species.remove('->')
    species.remove('E')
    species = ['e']+species
    species = ' '.join(species)

    file.write('ELEMENTS'+'\n')
    file.write('e C H'+'\n')
    file.write('END'+'\n')
    file.write('\n')
    file.write('SPECIES'+'\n')
    file.write(species+'\n')
    file.write('END'+'\n')


